In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

#!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

Libraries imported.


In [2]:
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Get the html from the wikipedia page with the postal codes of Canada

In [3]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

# List in Array

In [5]:
postalCodeList = []
boroughList = []
neighborhoodList = []

# Location

In [6]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [7]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

# Loading Pandas DataFrame

In [8]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


# Removing Non-Assigned Columns

In [9]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [10]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Question 1

In [12]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

test_df  
#print(test_df)

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


# Question 2

In [13]:
toronto_df_grouped.shape

(103, 3)

In [14]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


# Latitude & Longitude

In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Map & Markers

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [20]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [21]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# Defining Foursquare Credentials

In [23]:
CLIENT_ID = 'KDZBX2DHXOYFSSBL1LU4QZ1S31UEWJSDWHPUAAGF44TOJFZH' # your Foursquare ID
CLIENT_SECRET = 'MTD45CXTKNLNMBLH0HJ3WZNHEYHYN34SJJBADGGNVABBG5PH'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KDZBX2DHXOYFSSBL1LU4QZ1S31UEWJSDWHPUAAGF44TOJFZH
CLIENT_SECRET:MTD45CXTKNLNMBLH0HJ3WZNHEYHYN34SJJBADGGNVABBG5PH


In [24]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=KDZBX2DHXOYFSSBL1LU4QZ1S31UEWJSDWHPUAAGF44TOJFZH&client_secret=MTD45CXTKNLNMBLH0HJ3WZNHEYHYN34SJJBADGGNVABBG5PH&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [25]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1755, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Japango,43.655268,-79.385165,Sushi Restaurant
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Rolltation,43.654918,-79.387424,Japanese Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Karine's,43.653699,-79.390743,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Cafe Plenty,43.654571,-79.389450,Café


In [26]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,45,45,45,45,45,45
M4K,East Toronto,The Danforth West / Riverdale,45,45,45,45,45,45
M4L,East Toronto,India Bazaar / The Beaches West,45,45,45,45,45,45
M4M,East Toronto,Studio District,45,45,45,45,45,45
M4N,Central Toronto,Lawrence Park,45,45,45,45,45,45
M4P,Central Toronto,Davisville North,45,45,45,45,45,45
M4R,Central Toronto,North Toronto West,45,45,45,45,45,45
M4S,Central Toronto,Davisville,45,45,45,45,45,45
M4T,Central Toronto,Moore Park / Summerhill East,45,45,45,45,45,45


# Analysing Areas

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1755, 39)


,PostalCode,Borough,Neighborhoods,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 39)


,PostalCode,Borough,Neighborhoods,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
1,M4K,East Toronto,The Danforth West / Riverdale,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
2,M4L,East Toronto,India Bazaar / The Beaches West,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
3,M4M,East Toronto,Studio District,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
4,M4N,Central Toronto,Lawrence Park,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
5,M4P,Central Toronto,Davisville North,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
6,M4R,Central Toronto,North Toronto West,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
7,M4S,Central Toronto,Davisville,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
8,M4T,Central Toronto,Moore Park / Summerhill East,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,0.066667,0.022222,0.022222,0.022222,0.044444,0.022222,0.066667,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222


In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
1,M4K,East Toronto,The Danforth West / Riverdale,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
2,M4L,East Toronto,India Bazaar / The Beaches West,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
3,M4M,East Toronto,Studio District,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
4,M4N,Central Toronto,Lawrence Park,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
5,M4P,Central Toronto,Davisville North,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
6,M4R,Central Toronto,North Toronto West,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
7,M4S,Central Toronto,Davisville,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
8,M4T,Central Toronto,Moore Park / Summerhill East,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court


# Question 3: Clustering

In [30]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/Users/jhanviram/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [31]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court


In [32]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
21,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,43.648198,-79.379817,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
23,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
24,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
25,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
26,M5T,Downtown Toronto,Kensington Market / Chinatown / Grange Park,43.653206,-79.400049,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
27,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
20,M5K,Downtown Toronto,Toronto Dominion Centre / Design Exchange,43.647177,-79.381576,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court


# Cluster Checking

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
21,Downtown Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
22,Central Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
23,Central Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
24,Central Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
25,Downtown Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
26,Downtown Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
27,Downtown Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
20,Downtown Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court
28,Downtown Toronto,0,Art Gallery,Café,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Breakfast Spot,Dessert Shop,Gastropub,French Restaurant,Food Court


# Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Cluster 5

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


**Result: Most of the neighbourhoods fall under CLUSTER 1**